# Chapter 14
* Combining two inndependent abstractions into a single class is often a sign of weak design
* I guess we're making a smart pointer class

Handle classes that:
-copy their objects
-don't copy their objects


In [1]:
.rawInput

Using raw input


In [2]:
template <class T> class Handle {
public:
    Handle(): p(0) { }
    Handle(const Handle& s): p(0) { if (s.p) p = s.p->clone(); }
    Handle& operator=(const Handle&);
    ~Handle() { delete p; }

    Handle(T* t): p(t) { }

    operator bool() const { return p; }
    T& operator*() const;
    T* operator->() const;

private:
    T* p;
};

#include <stdexcept>

using std::runtime_error;

template <class T>
Handle<T>& Handle<T>::operator=(const Handle& rhs)
{
    if (&rhs != this) {
        delete p;
        p = rhs.p ? rhs.p->clone() : 0;
    }
    return *this;
}

template <class T>
T& Handle<T>::operator*() const
{
    if (p) 	
        return *p;
    throw runtime_error("unbound Handle");
}

template <class T>
T* Handle<T>::operator->() const
{
    if (p)
        return p;
    throw runtime_error("unbound Handle");
}


In [3]:
.rawInput

Not using raw input


for some general type x,
x->y
is equivalent to
(x.operator->())->y

similarly
(\*student).grade() calls grade() through a reference

Template Specializations look like template definitions they are specializing, but omit a few type parameters, replacing them with specific types instead

In [4]:
.rawInput

Using raw input


In [5]:
class Ref_count {
public:
  Ref_count(): count(new std::size_t(1)) { }

  std::size_t val() const { return *count; }
  void operator ++() const { ++*count; }
  void operator --() const { if (--*count == 0) delete count; }

private:
  std::size_t* count;
};

template <class T> class Ptr {
public:
    void make_unique() {
    if (refptr.val() != 1) {
      --refptr;
      refptr = Ref_count();
      p = p? clone(p): 0;
    }
  }

 Ptr(): p(0) { }
 Ptr(T* t): p(t) { }
 Ptr(const Ptr& h): p(h.p), refptr(h.refptr) { ++refptr; }

  Ptr& operator=(const Ptr&);
  ~Ptr();
  operator bool() const { return p; }
  T& operator*() const;
  T* operator->() const;

private:
  T* p;
  Ref_count refptr;
};

template <class T> T* clone(const T* tp) {
  return tp->clone();
}

template<class T> T& Ptr<T>::operator*() const { 
  if (p) return *p; throw std::runtime_error("unbound Ptr"); 
}

template<class T> T* Ptr<T>::operator->() const { 
  if (p) return p; throw std::runtime_error("unbound Ptr"); 
}

template<class T> Ptr<T>& Ptr<T>::operator=(const Ptr& rhs) {
  ++rhs.refptr;
  --refptr;

  if (refptr.val() == 0) {
    delete p;
  }

  refptr = rhs.refptr;
  p = rhs.p;

  return *this;
}

template<class T> Ptr<T>::~Ptr() {
    --refptr;

    if (refptr.val() == 0) {
        delete p;
    }
}

In [6]:
.rawInput

Not using raw input
